## German-English machine translations

The [OpenNMT](http://opennmt.net/OpenNMT-py/main.html) framework  is a open source neural machine translation system  designed for the reserach purpose for machine translation. The open source code is used in this notebook for translating German sentences to English automatically. 




Clone the source code from the Github repository  [OpenNMT-py](https://github.com/OpenNMT/OpenNMT-py)  and install the requirements.

In [1]:
!git clone https://github.com/OpenNMT/OpenNMT-py
%cd OpenNMT-py

Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 14399, done.
remote: Total 14399 (delta 0), reused 0 (delta 0), pack-reused 14399
Receiving objects: 100% (14399/14399), 145.88 MiB | 25.89 MiB/s, done.
Resolving deltas: 100% (10306/10306), done.
/content/OpenNMT-py


In [2]:
!pip install -r requirements.txt

  Cloning https://github.com/pytorch/text.git (to revision master) to /tmp/pip-req-build-l5rjfjz6
  Running command git clone -q https://github.com/pytorch/text.git /tmp/pip-req-build-l5rjfjz6
     |████████████████████████████████| 51kB 4.2MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-0.14.0-cp36-none-any.whl size=17503 sha256=26ed59a7ca6d8ec14eb90539f20c7331cca79c7d749730c3150478b91fd73730
  Stored in directory: /root/.cache/pip/wheels/aa/9c/ce/7e904dddb8c7595ffbe3409d24455bc5005852850e36011bda
  Created wheel for torchtext: filename=torchtext-0.4.0-cp36-none-any.whl size=53122 sha256=60deae4a1d466c3e16d1a79e61deae55e35f6212510bbc199f79e74794127367
  Stored in directory: /tmp/pip-ephem-wheel-cache-f3yx5jwf/wheels/47/f9/8d/a9e397ec2629a3fd3219b2ebc3ec8b55396fd3cf55963a77a5
Successfully built configargparse torchtext
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1
  Found existing installation: torc

The dataset used for DE-EN machine translation is the dataset provided for the WMT’16 multimodal machine translation task and can be found [here](http://www.statmt.org/wmt16/multimodal-task.html).

The zip files of the training, valiadtion and test sets are downloaded and extracted into data/multi30k folder.

In [0]:
!mkdir -p data/multi30k
!wget http://www.quest.dcs.shef.ac.uk/wmt16_files_mmt/training.tar.gz &&  tar -xf training.tar.gz -C data/multi30k && rm training.tar.gz
!wget http://www.quest.dcs.shef.ac.uk/wmt16_files_mmt/validation.tar.gz && tar -xf validation.tar.gz -C data/multi30k && rm validation.tar.gz
!wget http://www.quest.dcs.shef.ac.uk/wmt17_files_mmt/mmt_task1_test2016.tar.gz && tar -xf mmt_task1_test2016.tar.gz -C data/multi30k && rm mmt_task1_test2016.tar.gz

URL transformed to HTTPS due to an HSTS policy
--2019-06-30 13:30:58--  https://www.quest.dcs.shef.ac.uk/wmt16_files_mmt/training.tar.gz
Resolving www.quest.dcs.shef.ac.uk (www.quest.dcs.shef.ac.uk)... 143.167.8.76
Connecting to www.quest.dcs.shef.ac.uk (www.quest.dcs.shef.ac.uk)|143.167.8.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1207136 (1.2M) [application/x-gzip]
Saving to: ‘training.tar.gz’

training.tar.gz     100%[===================>]   1.15M   837KB/s    in 1.4s    

2019-06-30 13:31:00 (837 KB/s) - ‘training.tar.gz’ saved [1207136/1207136]

URL transformed to HTTPS due to an HSTS policy
--2019-06-30 13:31:00--  https://www.quest.dcs.shef.ac.uk/wmt16_files_mmt/validation.tar.gz
Resolving www.quest.dcs.shef.ac.uk (www.quest.dcs.shef.ac.uk)... 143.167.8.76
Connecting to www.quest.dcs.shef.ac.uk (www.quest.dcs.shef.ac.uk)|143.167.8.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46329 (45K) [application/x-gzip]
Saving

In [0]:
#preprocessing of the training and validation datasets using preprocess.py from OpenNMT

!for l in en de; do for f in data/multi30k/*.$l; do if [[ "$f" != *"test"* ]]; then sed -i "$ d" $f; fi;  done; done
!for l in en de; do for f in data/multi30k/*.$l; do perl tools/tokenizer.perl -a -no-escape -l $l -q  < $f > $f.atok; done; done
!python preprocess.py -train_src data/multi30k/train.de.atok -train_tgt data/multi30k/train.en.atok -valid_src data/multi30k/val.de.atok -valid_tgt data/multi30k/val.en.atok -save_data data/multi30k.atok.low -lower

[2019-06-30 13:31:29,190 INFO] Extracting features...
[2019-06-30 13:31:29,190 INFO]  * number of source features: 0.
[2019-06-30 13:31:29,190 INFO]  * number of target features: 0.
[2019-06-30 13:31:29,190 INFO] Building `Fields` object...
[2019-06-30 13:31:29,190 INFO] Building & saving training data...
[2019-06-30 13:31:29,190 INFO] Reading source and target files: data/multi30k/train.de.atok data/multi30k/train.en.atok.
[2019-06-30 13:31:29,198 INFO] Building shard 0.
[2019-06-30 13:31:30,176 INFO]  * saving 0th train data shard to data/multi30k.atok.low.train.0.pt.
[2019-06-30 13:31:31,179 INFO]  * tgt vocab size: 10843.
[2019-06-30 13:31:31,214 INFO]  * src vocab size: 18561.
[2019-06-30 13:31:31,311 INFO] Building & saving validation data...
[2019-06-30 13:31:31,312 INFO] Reading source and target files: data/multi30k/val.de.atok data/multi30k/val.en.atok.
[2019-06-30 13:31:31,312 INFO] Building shard 0.
[2019-06-30 13:31:31,326 INFO]  * saving 0th valid data shard to data/multi

In [0]:
# train the model using GPU 0 and save

!python train.py -data data/multi30k.atok.low -save_model multi30k_model -gpu_ranks 0

[2019-06-30 13:31:37,700 INFO]  * src vocab size = 18561
[2019-06-30 13:31:37,700 INFO]  * tgt vocab size = 10843
[2019-06-30 13:31:37,700 INFO] Building model...
[2019-06-30 13:31:45,517 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(18561, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(10843, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500, out_features=500, bias=False)
      (linear_out): Linear(in_featu

In [0]:
# list of the models saved at different training steps

!ls

available_models			  multi30k_model_step_5000.pt
CHANGELOG.md				  multi30k_model_step_55000.pt
config					  multi30k_model_step_60000.pt
CONTRIBUTING.md				  multi30k_model_step_65000.pt
data					  multi30k_model_step_70000.pt
Dockerfile				  multi30k_model_step_75000.pt
docs					  multi30k_model_step_80000.pt
floyd_requirements.txt			  multi30k_model_step_85000.pt
floyd.yml				  multi30k_model_step_90000.pt
github_deploy_key_opennmt_opennmt_py.enc  multi30k_model_step_95000.pt
LICENSE.md				  onmt
multi30k_model_step_100000.pt		  preprocess.py
multi30k_model_step_10000.pt		  README.md
multi30k_model_step_15000.pt		  requirements.opt.txt
multi30k_model_step_20000.pt		  requirements.txt
multi30k_model_step_25000.pt		  server.py
multi30k_model_step_30000.pt		  setup.py
multi30k_model_step_35000.pt		  tools
multi30k_model_step_40000.pt		  train.py
multi30k_model_step_45000.pt		  translate.py
multi30k_model_step_50000.pt


In [0]:
#to download models from colab to local system


from google.colab import files
files.download( "multi30k_model_step_95000.pt" )  

In [0]:
# translate the German testset of 1000 sentences into English language and the English reference translations are only used for finding the GOLD SCORE

!python translate.py -gpu 0 -model multi30k_model_*_100000.pt -src data/multi30k/test2016.de.atok -tgt data/multi30k/test2016.en.atok -replace_unk -verbose -output multi30k.test.pred.en.atok

[2019-06-30 15:32:51,791 INFO] Translating shard 0.

SENT 1: ['Ein', 'Mann', 'mit', 'einem', 'orangefarbenen', 'Hut', ',', 'der', 'etwas', 'anstarrt', '.']
PRED 1: A man in an orange hat welding something .
PRED SCORE: -2.2037
GOLD 1: A man in an orange hat starring at something .
GOLD SCORE: -22.7660

SENT 2: ['Ein', 'Boston', 'Terrier', 'läuft', 'über', 'saftig', '@-@', 'grünes', 'Gras', 'vor', 'einem', 'weißen', 'Zaun', '.']
PRED 2: A Boston Terrier dog runs on low murky grass in front of a white fence .
PRED SCORE: -2.6310
GOLD 2: A Boston Terrier is running on lush green grass in front of a white fence .
GOLD SCORE: -15.8407

SENT 3: ['Ein', 'Mädchen', 'in', 'einem', 'Karateanzug', 'bricht', 'ein', 'Brett', 'mit', 'einem', 'Tritt', '.']
PRED 3: A girl in a karate uniform is conducting a board with a kick .
PRED SCORE: -1.2818
GOLD 3: A girl in karate uniform breaking a stick with a front kick .
GOLD SCORE: -127.7019

SENT 4: ['Fünf', 'Leute', 'in', 'Winterjacken', 'und', 'mit', 'H